Apparently, we are lucky, and our predecessor already parsed out all addresses from god knows where. We can just fetch them with a scraper :)

In [181]:
import bs4
import urllib.request
from urllib.error import HTTPError
from datetime import date, timedelta
import pandas as pd

In [182]:
def decode_to_tuple(string):
    decoded_string = string[2:-1]
    entries = decoded_string.strip().split('\\r\\n')
    geocoded_addrs = []
    for entry in entries:
        entry = entry.strip('()')
        if entry == "": continue
        add, loc = entry.split("',")
        add = add[1:].encode("latin1").decode('unicode_escape')
        loc = [float(e) for e in loc.split(", ")]
        geocode = (add, loc[0], loc[1])
        geocoded_addrs.append(geocode)
    return geocoded_addrs

def daterange(start_date: date, end_date: date):
    days = int((end_date - start_date).days)
    for n in range(days):
        yield start_date + timedelta(n)

In [194]:
link = "https://qvwx.de/sperrmuell-bonn-2024/2024-01-06/Koordinaten.txt"

addresses = []

start_date = date(2024, 1, 1)
end_date = date(2024, 12, 31)
for single_date in daterange(start_date, end_date):
    d = single_date.strftime("%Y-%m-%d")
    link = f"https://qvwx.de/sperrmuell-bonn-2024/{d}/Koordinaten.txt"
    try:
        webpage=str(urllib.request.urlopen(link).read())
        soup = bs4.BeautifulSoup(webpage)
        tuples = decode_to_tuple(soup.get_text(strip=True))
        addresses.extend(tuples)
    except HTTPError as e:
        continue

In [195]:
df = pd.DataFrame(
    addresses, 
    columns=["Address", "LAT", "LON"]
    )
df

,Address,LAT,LON
0,"Am Agnesstift 1, 53117 Bonn",50.750893,7.088830
1,"Am Agnesstift 11, 53117 Bonn",50.750448,7.088701
2,"Am Agnesstift 13, 53117 Bonn",50.750567,7.089203
3,"Am Agnesstift 3, 53117 Bonn",50.750820,7.088642
4,"Am Agnesstift 5, 53117 Bonn",50.750701,7.088203
...,...,...,...
167545,"Weinheimstraße 58, 53229 Bonn",50.729465,7.185632
167546,"Weinheimstraße 6, 53229 Bonn",50.733607,7.180045
167547,"Weinheimstraße 7, 53229 Bonn",50.733337,7.180511
167548,"Weinheimstraße 8, 53229 Bonn",50.733411,7.179996


In [220]:
df[['street_part', 'city_part']] = df['Address'].str.split(',', expand=True)
df[['STRASSE1', 'HNR']] = df['street_part'].str.rsplit(' ', n=1, expand=True)
df['HNR'] = df['HNR'].astype(int)
df[['PLZ1', 'city']] = df['city_part'].str.strip().str.split(' ', n=1, expand=True)
df['PLZ1'] = df['PLZ1'].astype(int)
df.drop(columns=['street_part', 'city_part'], inplace=True)

df

,Address,LAT,LON,STRASSE1,HNR,PLZ1,city
0,"Am Agnesstift 1, 53117 Bonn",50.750893,7.088830,Am Agnesstift,1,53117,Bonn
1,"Am Agnesstift 11, 53117 Bonn",50.750448,7.088701,Am Agnesstift,11,53117,Bonn
2,"Am Agnesstift 13, 53117 Bonn",50.750567,7.089203,Am Agnesstift,13,53117,Bonn
3,"Am Agnesstift 3, 53117 Bonn",50.750820,7.088642,Am Agnesstift,3,53117,Bonn
4,"Am Agnesstift 5, 53117 Bonn",50.750701,7.088203,Am Agnesstift,5,53117,Bonn
...,...,...,...,...,...,...,...
167545,"Weinheimstraße 58, 53229 Bonn",50.729465,7.185632,Weinheimstraße,58,53229,Bonn
167546,"Weinheimstraße 6, 53229 Bonn",50.733607,7.180045,Weinheimstraße,6,53229,Bonn
167547,"Weinheimstraße 7, 53229 Bonn",50.733337,7.180511,Weinheimstraße,7,53229,Bonn
167548,"Weinheimstraße 8, 53229 Bonn",50.733411,7.179996,Weinheimstraße,8,53229,Bonn


In [221]:
df.to_csv("/home/bjorge_local/project_src/spermull-bonn/data/geocoded_addr.csv")